In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
spark = SparkSession.builder.appName('InserindoMysql')\
.getOrCreate()
spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


# --> Construindo a tabela de PIB/População por estado

In [7]:
dados_ibge = spark.read.parquet('gs://parquetcassandranatal/Dados_ibge')

In [8]:
dados_ibge.show(3)

+--------+---+--------------+---------+----+-------------+------+-------------+
|cod_ibge| uf|nome_municipio|populacao| ano|cod_municipio|cod_uf|pib_municipio|
+--------+---+--------------+---------+----+-------------+------+-------------+
| 1303304| AM| Novo Aripuanã|    26443|2018|         3304|    13|    199377509|
| 1303304| AM| Novo Aripuanã|    26443|2017|         3304|    13|    183333826|
| 1304260| AM|        Uarini|    13839|2018|         4260|    13|    187766213|
+--------+---+--------------+---------+----+-------------+------+-------------+
only showing top 3 rows



In [9]:
dados_ibge.printSchema()

root
 |-- cod_ibge: integer (nullable = true)
 |-- uf: string (nullable = true)
 |-- nome_municipio: string (nullable = true)
 |-- populacao: integer (nullable = true)
 |-- ano: integer (nullable = true)
 |-- cod_municipio: integer (nullable = true)
 |-- cod_uf: integer (nullable = true)
 |-- pib_municipio: long (nullable = true)



In [12]:
pib17 = dados_ibge.groupBy('uf','ano') \
    .agg(sum('pib_municipio').astype(LongType()).alias('pib'),
        sum('populacao').alias('populacao')) \
    .orderBy(col('pib').desc()).filter(col('ano')==2017)
pib17.show(4) 

+---+----+-------------+---------+
| uf| ano|          pib|populacao|
+---+----+-------------+---------+
| SP|2017|2120761635108| 46649132|
| RJ|2017| 671605668055| 17463349|
| MG|2017| 576375544695| 21411923|
| RS|2017| 423270047295| 11466630|
+---+----+-------------+---------+
only showing top 4 rows



In [13]:
pib18 = dados_ibge.groupBy('uf','ano') \
    .agg(sum('pib_municipio').astype(LongType()).alias('pib'),
        sum('populacao').alias('populacao')) \
    .orderBy(col('pib').desc()).filter(col('ano')==2018)
pib18.show(4) 

+---+----+-------------+---------+
| uf| ano|          pib|populacao|
+---+----+-------------+---------+
| SP|2018|2210561949477| 46649132|
| RJ|2018| 758859046867| 17463349|
| MG|2018| 614875819804| 21411923|
| RS|2018| 457293957585| 11466630|
+---+----+-------------+---------+
only showing top 4 rows



In [14]:
pib17 = pib17.toPandas()

In [15]:
pib18 = pib18.toPandas()

In [18]:
pib17.to_parquet('gs://parquetapresentacao/17ibge')
pib18.to_parquet('gs://parquetapresentacao/18ibge')

# ->  Construindo tabela cobertura por operadora

In [2]:
reclamacao = spark.read.parquet('gs://parquetcassandranatal/Reclamacao')

In [3]:
reclamacao.count()

2590318

In [4]:
reclamacao = reclamacao.filter((col('marca')=='CLARO') |
                  (col('marca')=='VIVO') |
                  (col('marca')=='TIM') |
                  (col('marca')=='OI'))

In [5]:
reclamacao = reclamacao.toPandas()

In [6]:
reclamacao['assunto'].unique()

array(['Cobrança', 'Qualidade, Funcionamento e Reparo',
       'Oferta, Bônus, Promoções, Mensagens Publicitárias e Mudança de Plano',
       'Bloqueio ou Suspensão e Desbloqueio',
       'Instalação ou Ativação ou Habilitação', 'Crédito Pré-pago',
       'Documento de Cobrança, Contrato, Gravações', 'Cancelamento',
       'Ressarcimento', 'Atendimento', 'Mudança de Endereço',
       'Portabilidade', 'Dados cadastrais e número da linha',
       'Longa Distância/Interurbano',
       'Programa Banda Larga nas Escolas (PBLE)', 'Pré-Pago',
       'Banda larga - ADSL', 'Pós-Pago', 'NÃO APLICÁVEL', 'TV a Cabo',
       'Local', 'Atuação Anatel'], dtype=object)

In [5]:
reclamacao.show(3)

+----+---+---+--------+-------------+----------------+-----+--------------------+
| ano|mes| uf|  cidade|cod_municipio|           canal|marca|             assunto|
+----+---+---+--------+-------------+----------------+-----+--------------------+
|2018| 12| MG|Pitangui|        51404|     Call Center|  TIM|            Cobrança|
|2018| 12| MG|  Piumhi|        51503|     Call Center|   OI|Qualidade, Funcio...|
|2018| 12| MG|  Piumhi|        51503|Aplicativo Móvel| VIVO|Oferta, Bônus, Pr...|
+----+---+---+--------+-------------+----------------+-----+--------------------+
only showing top 3 rows



In [6]:
reclamacao.dropna()

DataFrame[ano: int, mes: int, uf: string, cidade: string, cod_municipio: int, canal: string, marca: string, assunto: string]

In [46]:
recmes = reclamacao.groupBy('marca','mes','uf') \
    .count().orderBy('marca','mes')
recmes.filter(col('uf')=='SP').show() 

+-----+---+---+-----+
|marca|mes| uf|count|
+-----+---+---+-----+
|CLARO|  1| SP| 9560|
|CLARO|  2| SP| 8660|
|CLARO|  3| SP| 9820|
|CLARO|  4| SP| 9536|
|CLARO|  5| SP| 9493|
|CLARO|  6| SP| 8248|
|CLARO|  7| SP| 8951|
|CLARO|  8| SP| 9409|
|CLARO|  9| SP| 8020|
|CLARO| 10| SP| 8647|
|CLARO| 11| SP| 8747|
|CLARO| 12| SP|11025|
|   OI|  1| SP| 3273|
|   OI|  2| SP| 2596|
|   OI|  3| SP| 2978|
|   OI|  4| SP| 2904|
|   OI|  5| SP| 3008|
|   OI|  6| SP| 2562|
|   OI|  7| SP| 2640|
|   OI|  8| SP| 3021|
+-----+---+---+-----+
only showing top 20 rows

